In [ ]:
import pandas as pd

# Cargar el archivo CSV y limpiar espacios en nombres de columnas
file_path = '/content/Base de datos Chiloé Silvestre.csv'
data = pd.read_csv(file_path).rename(columns=lambda x: x.strip())

# Convertir la primera letra a mayúscula en los nombres de columnas excepto en 'Fecha de ingreso'
data.columns = [col.capitalize() if col != 'Fecha de ingreso' else col for col in data.columns]

# Convertir el DataFrame a un diccionario con todas las filas y columnas
diccionario_completo = data.to_dict(orient='records')

# Separar en dos diccionarios: uno con fechas válidas y otro con 'No data'
datos_con_fechas = []
datos_sin_fechas = []

for fila in diccionario_completo:
    if fila['Fecha de ingreso'] != 'No data':
        fila['Fecha de ingreso'] = pd.to_datetime(fila['Fecha de ingreso'], errors='coerce').strftime('%d/%m/%Y')
        datos_con_fechas.append(fila)
    else:
        datos_sin_fechas.append(fila)

# Crear el diccionario datos_completos fusionando ambos diccionarios
datos_completos = datos_con_fechas + datos_sin_fechas

# Mostrar los primeros 5 registros del diccionario completo como verificación (opcional)
print(datos_completos[:5])
data.head()

[{'Nombre común': 'Cisne de cuello negro', 'Edad': 'Adulto', 'Procedencia': 'No data', 'Fecha de ingreso': '26/07/2021', 'Motivo de ingreso': 'Ataque de perros', 'Destino': 'Muerte'}, {'Nombre común': 'Gato Güiña', 'Edad': 'Adulto', 'Procedencia': 'Ancud', 'Fecha de ingreso': '21/12/2022', 'Motivo de ingreso': 'Atropello', 'Destino': 'Liberación'}, {'Nombre común': 'Lobo marino común', 'Edad': 'Juvenil', 'Procedencia': 'Ancud', 'Fecha de ingreso': '23/06/2022', 'Motivo de ingreso': 'Ataque de perros', 'Destino': 'Liberación'}, {'Nombre común': 'Pudú', 'Edad': 'Adulto Juvenil', 'Procedencia': 'Ancud', 'Fecha de ingreso': '31/12/2020', 'Motivo de ingreso': 'Ataque de perros', 'Destino': 'Liberación'}, {'Nombre común': 'Pudú', 'Edad': 'Adulto', 'Procedencia': 'Ancud', 'Fecha de ingreso': '05/09/2022', 'Motivo de ingreso': 'Ataque de perros', 'Destino': 'Liberación'}]


,Nombre común,Edad,Procedencia,Fecha de ingreso,Motivo de ingreso,Destino
0,Cisne de cuello negro,Adulto,No data,7/26/2021,Ataque de perros,Muerte
1,Gato Güiña,Adulto,Ancud,12/21/2022,Atropello,Liberación
2,Lobo marino común,Juvenil,Ancud,6/23/2022,Ataque de perros,Liberación
3,Pudú,Adulto Juvenil,Ancud,12/31/2020,Ataque de perros,Liberación
4,Pudú,Adulto,Ancud,9/5/2022,Ataque de perros,Liberación


In [ ]:
#HALLAZGO 1:
import plotly.express as px
import pandas as pd

# Filtrar datos específicamente para el primer hallazgo
# Convertir el diccionario completo a un DataFrame para facilitar el filtrado y la visualización
df = pd.DataFrame(datos_completos)

# Filtrar los pudúes ingresados por ataques de perros
filtro_pudues = (df['Nombre común'].str.lower() == 'pudú') & (df['Motivo de ingreso'].str.lower() == 'ataque de perros')

# Contar total de pudúes ingresados por ataques de perros
total_pudues = df[filtro_pudues].shape[0]

# Contar los que sobrevivieron y los que no
sobrevivieron = df[filtro_pudues]['Destino'].str.lower().isin(['liberación', 'rehabilitación']).sum()
no_sobrevivieron = total_pudues - sobrevivieron

# Crear un diccionario con los datos para el gráfico de donas
datos_hallazgo1 = {
    'Destino': ['Sobrevivieron', 'No sobrevivieron'],
    'Cantidad': [sobrevivieron, no_sobrevivieron]
}

# Crear el DataFrame a partir del diccionario
df_hallazgo1 = pd.DataFrame(datos_hallazgo1)

# Crear el gráfico de donas con Plotly Express
fig = px.pie(df_hallazgo1, values='Cantidad', names='Destino', title='Destino de pudúes ingresados por ataques de perros',
             color_discrete_sequence=px.colors.qualitative.Set3)

# Mostrar el gráfico
fig.show()


In [ ]:
#HALLAZGO 2
import pandas as pd
import plotly.graph_objects as go

# Filtrar datos para obtener solo los animales ingresados por ataques de perros
animales_por_ataque_perro = [fila for fila in datos_completos if fila['Motivo de ingreso'] == 'Ataque de perros']

# Contar los tipos de animales atacados por perros, agrupando los que no son pudúes como "Resto de animales"
tipos_animales = {'Pudú': 0, 'Resto de animales': 0}
for fila in animales_por_ataque_perro:
    tipo_animal = fila['Nombre común']
    if tipo_animal == 'Pudú':
        tipos_animales['Pudú'] += 1
    else:
        tipos_animales['Resto de animales'] += 1

# Crear listas separadas para nombres de animales y número de ataques
nombres_animales = list(tipos_animales.keys())
num_ataques = list(tipos_animales.values())

# Crear el gráfico de dona
fig = go.Figure(data=[go.Pie(labels=nombres_animales, values=num_ataques, hole=0.5)])

fig.update_layout(
    title='Proporción de animales atacados por perros en Chiloé Silvestre (2017-2024)',
    showlegend=True,
)

# Actualizar la leyenda para especificar "Resto de animales"
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=15,
                  marker=dict(colors=['gold', 'lightgreen', 'darkorange', 'crimson'],
                              line=dict(color='#000000', width=2)))

fig.show()


In [ ]:
#HALLAZGO 3.1
import plotly.graph_objects as go

# Crear lista de animales ingresados ordenados por fecha de ingreso y filtrados por el período 2020-2022
animales_ingresados = []
for fila in datos_completos:
    fecha_ingreso = fila['Fecha de ingreso']
    if fecha_ingreso != 'No data':
        try:
            fecha_dt = pd.to_datetime(fecha_ingreso, format='%d/%m/%Y')
            if fecha_dt.year >= 2017 and fecha_dt.year <= 2024:
                animales_ingresados.append({
                    'Nombre del animal': fila['Nombre común'],
                    'Fecha de ingreso': fecha_dt,
                    'Número de animales': 1,
                    'Número de pudúes': 1 if fila['Nombre común'] == 'Pudú' else 0
                })
        except ValueError:
            continue

# Crear DataFrame con los datos ordenados por fecha de ingreso
df_animales_ingresados = pd.DataFrame(animales_ingresados).sort_values(by='Fecha de ingreso')

# Calcular porcentaje acumulado de pudúes respecto al total de animales ingresados
df_animales_ingresados['% Pudúes/Animales'] = (df_animales_ingresados['Número de pudúes'].cumsum() / df_animales_ingresados['Número de animales'].cumsum()) * 100

# Crear gráfico de línea con Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_animales_ingresados['Fecha de ingreso'],
                         y=df_animales_ingresados['% Pudúes/Animales'],
                         mode='lines',
                         name='% Pudúes/Animales',
                         line=dict(color='blue', width=2),
                         hovertemplate='<b>%{x}</b><br>'
                                       '<i>% Pudúes/Animales:</i> %{y:.2f}%<extra></extra>'
                        ))

# Configurar el layout del gráfico
fig.update_layout(
    title='Porcentaje acumulado de Pudúes respecto al total de animales ingresados (2017-2024)',
    xaxis_title='Fecha de ingreso',
    yaxis_title='% Pudúes/Animales',
    hovermode='x unified'
)

# Mostrar el gráfico
fig.show()


In [ ]:
#HALLAZGO 3.1 ALT
import pandas as pd
import plotly.graph_objects as go
import numpy as np
from sklearn.linear_model import LinearRegression

# Filtrar datos para obtener todos los animales ingresados con fecha válida desde 2020 hasta 2022
animales_ingresados = []
for fila in datos_completos:
    fecha_ingreso = fila['Fecha de ingreso']
    if fecha_ingreso != 'No data':
        try:
            fecha_dt = pd.to_datetime(fecha_ingreso, format='%d/%m/%Y')
            if fecha_dt.year >= 2021 and fecha_dt.year <= 2022:
                animales_ingresados.append({'Nombre común': fila['Nombre común'], 'Fecha de ingreso': fecha_dt})
        except ValueError:
            continue

# Filtrar datos para obtener solo los pudúes ingresados con fecha válida desde 2020 hasta 2022
pudues_ingresados = []
for fila in datos_completos:
    if fila['Nombre común'] == 'Pudú':
        fecha_ingreso = fila['Fecha de ingreso']
        if fecha_ingreso != 'No data':
            try:
                fecha_dt = pd.to_datetime(fecha_ingreso, format='%d/%m/%Y')
                if fecha_dt.year >= 2021 and fecha_dt.year <= 2022:
                    pudues_ingresados.append({'Nombre común': 'Pudú', 'Fecha de ingreso': fecha_dt})
            except ValueError:
                continue

# Crear DataFrames para animales y pudúes ingresados
df_animales = pd.DataFrame(animales_ingresados)
df_pudues = pd.DataFrame(pudues_ingresados)

# Contar ingresos por día para animales y pudúes
ingresos_animales = df_animales.groupby(df_animales['Fecha de ingreso'].dt.date).size().cumsum()
ingresos_pudues = df_pudues.groupby(df_pudues['Fecha de ingreso'].dt.date).size().cumsum()

# Función para calcular pendiente dinámica hasta un punto específico
def calcular_pendiente_dinamica(x, y, indice_punto):
    x = np.array(x[:indice_punto]).reshape(-1, 1)
    y = np.array(y[:indice_punto]).reshape(-1, 1)
    modelo = LinearRegression()
    modelo.fit(x, y)
    return modelo.coef_[0][0]

# Crear figura de Plotly
fig = go.Figure()

# Añadir trazas para animales y pudúes con pendientes dinámicas
for nombre, datos, color in [('Animales ingresados', ingresos_animales, 'blue'), ('Pudúes ingresados', ingresos_pudues, 'green')]:
    pendientes_dinamicas = []
    for i in range(len(datos)):
        pendiente = calcular_pendiente_dinamica(range(i+1), datos.values, i+1)
        pendientes_dinamicas.append(pendiente)

    fig.add_trace(go.Scatter(x=datos.index, y=datos.values,
                             mode='lines+markers',
                             name=f'{nombre}',
                             line=dict(color=color),
                             hovertemplate='<b>%{y}</b><br>'
                                           '<i>Pendiente:</i> %{customdata:.2f}<extra></extra>',
                             customdata=pendientes_dinamicas))

# Configurar layout del gráfico
fig.update_layout(
    title='Evolución diaria de ingresos de animales y pudúes en Chiloé Silvestre (2021-2022)',
    xaxis_title='Fecha de ingreso',
    yaxis_title='Total acumulado de ingresos',
    legend=dict(x=0.7, y=1.1),
    hovermode='x unified'
)

# Mostrar gráfico
fig.show()


In [ ]:
#ANEXO 3.1

#HALLAZGO 3.1 ALT
import pandas as pd
import plotly.graph_objects as go
import numpy as np
from sklearn.linear_model import LinearRegression


# Filtrar datos para obtener solo los pudúes ingresados con fecha válida desde 2021 hasta 2022
pudues_ingresados = []
for fila in datos_completos:
    if fila['Nombre común'] == 'Pudú':
        fecha_ingreso = fila['Fecha de ingreso']
        if fecha_ingreso != 'No data':
            try:
                fecha_dt = pd.to_datetime(fecha_ingreso, format='%d/%m/%Y')
                if fecha_dt.year >= 2021 and fecha_dt.year <= 2022:
                    pudues_ingresados.append({'Nombre común': 'Pudú', 'Fecha de ingreso': fecha_dt})
            except ValueError:
                continue

# Filtrar datos para obtener solo los pudúes ingresados por ataques de perros con fecha válida desde 2021 hasta 2022
pudues_ingresados_perros = []
for fila in datos_completos:
    if fila['Nombre común'] == 'Pudú' and fila['Motivo de ingreso'] == 'Ataque de perros':
        fecha_ingreso = fila['Fecha de ingreso']
        if fecha_ingreso != 'No data':
            try:
                fecha_dt = pd.to_datetime(fecha_ingreso, format='%d/%m/%Y')
                if fecha_dt.year >= 2021 and fecha_dt.year <= 2022:
                    pudues_ingresados_perros.append({'Nombre común': 'Pudú', 'Fecha de ingreso': fecha_dt})
            except ValueError:
                continue

# Crear DataFrames para pudúes ingresados y pudúes ingresados por ataques de perros
df_pudues = pd.DataFrame(pudues_ingresados)
df_pudues_perros = pd.DataFrame(pudues_ingresados_perros)

# Contar ingresos por día para pudúes y pudúes por ataques de perros
ingresos_pudues = df_pudues.groupby(df_pudues['Fecha de ingreso'].dt.date).size().cumsum()
ingresos_pudues_perros = df_pudues_perros.groupby(df_pudues_perros['Fecha de ingreso'].dt.date).size().cumsum()

# Función para calcular pendiente dinámica hasta un punto específico
def calcular_pendiente_dinamica(x, y, indice_punto):
    x = np.array(x[:indice_punto]).reshape(-1, 1)
    y = np.array(y[:indice_punto]).reshape(-1, 1)
    modelo = LinearRegression()
    modelo.fit(x, y)
    return modelo.coef_[0][0]

# Crear figura de Plotly
fig = go.Figure()

# Añadir trazas para pudúes y pudúes por ataques de perros con pendientes dinámicas
for nombre, datos, color in [('Pudúes ingresados', ingresos_pudues, 'blue'), ('Pudúes ingresados por ataques de perros', ingresos_pudues_perros, 'green')]:
    pendientes_dinamicas = []
    for i in range(len(datos)):
        pendiente = calcular_pendiente_dinamica(range(i+1), datos.values, i+1)
        pendientes_dinamicas.append(pendiente)

    fig.add_trace(go.Scatter(x=datos.index, y=datos.values,
                             mode='lines+markers',
                             name=f'{nombre}',
                             line=dict(color=color),
                             hovertemplate='<b>%{y}</b><br>'
                                           '<i>Pendiente:</i> %{customdata:.2f}<extra></extra>',
                             customdata=pendientes_dinamicas))

# Configurar layout del gráfico
fig.update_layout(
    title='Evolución diaria de ingresos de pudúes y pudúes por ataques de perros en Chiloé Silvestre (2021-2022)',
    xaxis_title='Fecha de ingreso',
    yaxis_title='Total acumulado de ingresos',
    legend=dict(x=0.7, y=1.1),
    hovermode='x unified'
)

# Mostrar gráfico
fig.show()

In [ ]:
#HALLAZGO 4:
import plotly.graph_objects as go

# Filtrar pudúes con destino "Muerte"
pudues_muertos = [fila for fila in datos_completos if fila['Destino'] == 'Muerte' and fila['Nombre común'] == 'Pudú']
# Contadores iniciales
count_ataque_perros = 0
count_atropello = 0
count_causas_naturales = 0

# Contar cada motivo de ingreso
for pudu in pudues_muertos:
    motivo = pudu['Motivo de ingreso']
    if motivo == 'Ataque de perros':
        count_ataque_perros += 1
    elif motivo == 'Atropello':
        count_atropello += 1
    elif motivo != 'Ataque de perros' or motivo!= 'Atropello':
        count_causas_naturales += 1

# Crear el gráfico de radar
fig = go.Figure()

fig.add_trace(go.Scatterpolar(
    r=[count_causas_naturales, count_atropello, count_ataque_perros],
    theta=['Muerte por causas naturales', 'Muerte por atropello', 'Muerte por ataques de perros'],
    fill='toself',
    name='Causas de muerte de Pudúes',
    mode='lines+markers',
    marker=dict(size=8),
    line=dict(width=1),
    subplot='polar'
))

fig.update_layout(
    polar=dict(radialaxis=dict(visible=True, showticklabels=True)),
    showlegend=True,
    title='Causas de muerte de Pudúes (Muerte)',
    width=700,
    height=500
)

fig.show()

In [ ]:
#HALLAZGO 7
import plotly.express as px

# Contar los animales totales de cada tipo
total_animales = len(datos_completos)
conteo_animales = {}
for fila in datos_completos:
    nombre_comun = fila['Nombre común']
    if nombre_comun in conteo_animales:
        conteo_animales[nombre_comun] += 1
    else:
        conteo_animales[nombre_comun] = 1

# Calcular el porcentaje de cada tipo de animal respecto al total
porcentaje_animales = {animal: (conteo / total_animales) * 100 for animal, conteo in conteo_animales.items()}

# Crear DataFrame para el gráfico de burbujas
df = pd.DataFrame({
    'Animal': list(porcentaje_animales.keys()),
    'Porcentaje': list(porcentaje_animales.values()),
    'Total': list(conteo_animales.values())
})

# Convertir la primera letra del nombre común a mayúscula
df['Animal'] = df['Animal'].str.capitalize()

# Ordenar DataFrame por el porcentaje en orden ascendente
df = df.sort_values(by='Porcentaje')

# Crear gráfico de burbujas con escala de colores
fig = px.scatter(df, x='Animal', y='Total', size='Porcentaje', color='Porcentaje',
                 hover_name='Animal', size_max=60,
                 labels={'Animal': 'Nombre común del animal', 'Total': 'Cantidad total de animales ingresados',
                         'Porcentaje': 'Porcentaje respecto al total (%)'},
                 title='Porcentaje de cada tipo de animal ingresado')

# Mostrar gráfico
fig.show()


In [ ]:
import plotly.graph_objects as go

# Crear un diccionario con la procedencia de cada ingreso de pudú
procedencia_pudues = [fila['Procedencia'] for fila in datos_completos if fila['Nombre común'] == 'Pudú']

# Contar la cantidad de pudúes para cada procedencia
conteo_procedencia = {}
for procedencia in procedencia_pudues:
    if procedencia in conteo_procedencia:
        conteo_procedencia[procedencia] += 1
    else:
        conteo_procedencia[procedencia] = 1

# Ordenar el diccionario por cantidad
conteo_procedencia = dict(sorted(conteo_procedencia.items(), key=lambda item: item[1]))

# Crear listas para las etiquetas, fuentes y destinos
labels = list(conteo_procedencia.keys()) + ["Total Ingresos"]
sources = list(range(len(conteo_procedencia)))
targets = [len(conteo_procedencia)] * len(conteo_procedencia)
values = list(conteo_procedencia.values())

# Crear el gráfico Sankey
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
        color="blue"
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values,
        color="rgba(0, 128, 255, 0.8)"
    )
)])

fig.update_layout(
    title_text="Procedencia de Ingresos de Pudúes",
    font_size=10
)

fig.show()
